# Part 05: Tabulate statistics on the street disconntinuties
michael babb  
2024 11 24

In [ ]:
# standard
import os

In [ ]:
# external
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# custom
import run_constants as rc
from utils import *

# output excel file

In [ ]:
ofpn = os.path.join(rc.ANALYSIS_OUTPUT_FILE_PATH, rc.S05_ANALYSIS_OUT_FILE_NAME)

In [ ]:
e_writer = pd.ExcelWriter(path = ofpn)

# load the street network data

In [ ]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S05_MISSING_IN_FILE_NAME)

In [ ]:
gdf = gpd.read_file(filename = fpn)

In [ ]:
gdf.columns

In [ ]:
gdf['n_segments'] = int(1)

In [ ]:
# add text descriptions for the street type categories
street_status_dict = {0:'continuous_street',
                1:'discontinuous_street',
                2:'added_street: within-sector',
                3:'added_street: cross-sector'}

In [ ]:
gdf['street_status_desc'] = gdf['street_status'].map(street_status_dict)

In [ ]:
# number of segments
gdf['street_status_desc'].value_counts()

In [ ]:
# simple count of road miles
col_names = ['street_status_desc', 'dist_miles']
gdf_agg = gdf[col_names].groupby(col_names[:-1]).agg(n_miles = ('dist_miles', 'sum')).reset_index()

In [ ]:
# total road miles
gdf_agg.head()

In [ ]:
col_names = ['ord_street_name', 'ord_stname_concat', 'street_status_desc', 'n_segments']
gdf_agg = gdf[col_names].drop_duplicates().groupby(col_names[2:-1]).agg(n_roads = ('n_segments', 'sum')).reset_index()

In [ ]:
gdf_agg.head()

In [ ]:
# number of added segment
gdf['street_status_desc'].value_counts()

In [ ]:
# average number of segments added per uniquely named road
3617 / 1103

# investigate if the cross-sector added streets are correct

In [ ]:
gdf.columns

In [ ]:
col_names = ['ord_stname_type_group', 'street_status', 'n_segments']

In [ ]:
# pivot
gdf_agg = pd.pivot_table(data = gdf, values = 'n_segments', index = ['ord_stname_type_group'],
                         columns = ['street_status'], aggfunc='sum', fill_value=0)

In [ ]:
# rename columns
gdf_agg.columns = ['ss_' + str(x) for x in range(0, 4)]

In [ ]:
gdf_agg = gdf_agg.reset_index()

In [ ]:
# perform a series of tests to identify the multi_section street
gdf_agg['is_multi_section'] = int(0)

In [ ]:
gdf_agg.loc[gdf_agg['ord_stname_type_group'].str.find('_') > 0, 'is_multi_section'] = int(1)

In [ ]:
# false negatives: an added segment is not marked cross-sector when it should be
gdf_agg['check_street_fn'] = int(0)

In [ ]:
gdf_agg.loc[(gdf_agg['is_multi_section'] == 1) &
            (gdf_agg['ss_3'] == 0), 'check_street_fn'] = int(1)

In [ ]:
gdf_agg['check_street_fn'].sum()

In [ ]:
# false positives: an added segment is marked cross-sector when it should not be
gdf_agg['check_street_fp'] = int(0)
gdf_agg.loc[(gdf_agg['is_multi_section'] == 0) &
            (gdf_agg['ss_3'] != 0), 'check_street_fp'] = int(1)

In [ ]:
gdf_agg['check_street_fp'].sum()

In [ ]:
# we only have false negatives. Let's highlight those

In [ ]:
check_gdf_agg = gdf_agg.loc[(gdf_agg['check_street_fn'] == 1) |
                      (gdf_agg['check_street_fp'] == 1), :]

In [ ]:
# export these streets
check_gdf = gdf.loc[gdf['ord_stname_type_group'].isin(check_gdf_agg['ord_stname_type_group']), :]

In [ ]:
check_gdf['ord_stname_type_group'].unique().shape

In [ ]:
# visualize
write_gdf(gdf = check_gdf, output_file_path=rc.OUTPUT_FILE_PATH, output_file_name='check_connections.gpkg')

In [ ]:
# okay, after looking at these segments in qGIS - they appear to be correct. 
# this is because the existing street crosses from one sector of the city to 
# another, but the added segments are all within-sector. So in a way, by a
# strict definition, these are false negatives, but these streets are correct. 

# what street name - excluding direction - has the most street types?

In [ ]:
col_names = ['ord_street_name', 'ord_street_type', 'n_segments']

In [ ]:
gdf_agg = gdf[col_names].drop_duplicates()

In [ ]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = 'ord_street_name', columns = ['ord_street_type'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [ ]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.head()

In [ ]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 's_type_count', index = False)

# what street name - excluding type - has the most street connections?

In [ ]:
col_names = ['ord_street_name', 'ord_stname_concat', 'street_status_desc', 'n_segments']
gdf_agg = gdf[col_names].drop_duplicates()

In [ ]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = 'ord_street_name', columns = ['street_status_desc'],
                        aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [ ]:
gdf_pv.head()

In [ ]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.columns

In [ ]:
gdf_pv.head()

In [ ]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'connection_count_by_name', index = False)

# what street name - including direction and type - has the most street connections?

In [ ]:
col_names = ['ord_stname_concat', 'ord_street_type', 'street_status', 'street_status_desc', 'n_segments']
gdf_agg = gdf[col_names]

In [ ]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['n_segments'], index = ['ord_stname_concat', 'ord_street_type'],
                        columns = ['street_status_desc'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [ ]:
gdf_pv.columns = [''.join(cn).replace('n_segments', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.head()

In [ ]:
# remove the all row
gdf_pv = gdf_pv.loc[gdf_pv['ord_stname_concat'] != 'All', :]

In [ ]:
gdf_pv.shape

# what are the streets with at least 10 discontinuities?

In [ ]:
gdf_pv.loc[gdf_pv['added_street'] >= 10, 'ord_stname_concat'].tolist()

In [ ]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'connection_count_full_name', index = False)

# what is the ratio of discontinuous to continuous streets?

In [ ]:
# number of streets with at least one added segment
split_streets = gdf_pv.loc[gdf_pv['discontinuous_street'] > 0, :].shape[0]
split_streets

In [ ]:
# number of streets with no added segments
complete_streets = gdf_pv.loc[gdf_pv['continuous_street'] > 0, :].shape[0]
complete_streets

In [ ]:
split_streets / gdf_pv.shape[0]

In [ ]:
complete_streets / gdf_pv.shape[0]

In [ ]:
# 44 percent of streets are multi-segments.
# 55 percent of streets are single segment.

# sum of road miles by discontinuous street

In [ ]:
col_names = ['ord_stname_concat', 'ord_street_type', 'street_status', 'street_status_desc', 'dist_miles']
gdf_agg = gdf[col_names]

In [ ]:
gdf_pv = pd.pivot_table(data = gdf_agg, values = ['dist_miles'], index = ['ord_stname_concat', 'ord_street_type'],
                        columns = ['street_status_desc'], aggfunc = 'sum', fill_value = 0, margins = True).reset_index()

In [ ]:
gdf_pv.columns = [''.join(cn).replace('dist_miles', '') for cn in gdf_pv.columns]

In [ ]:
gdf_pv.shape

In [ ]:
gdf_pv = gdf_pv.loc[gdf_pv['ord_stname_concat']!= 'All', :].copy()

In [ ]:
gdf_pv.shape

In [ ]:
gdf_pv.head()

# which streets feature longer discontinuous portions?

In [ ]:
gdf_pv['longer_added'] = int(0) # street is complete
gdf_pv.loc[(gdf_pv['continuous_street'] == 0) & (gdf_pv['added_street'] <= gdf_pv['discontinuous_street']), 'longer_added'] = int(1)
gdf_pv.loc[(gdf_pv['continuous_street'] == 0) & (gdf_pv['added_street'] > gdf_pv['discontinuous_street']), 'longer_added'] = int(2)

In [ ]:
gdf_pv['longer_added'].value_counts()

In [ ]:
gdf_pv.to_excel(excel_writer=e_writer, sheet_name = 'distance_full_name', index = False)

In [ ]:
# road miles on streets with at least one added segment
split_streets = gdf_pv['discontinuous_street'].sum()
split_streets

In [ ]:
# road miles on streets with no added segments
complete_streets = gdf_pv['continuous_street'].sum()
complete_streets

In [ ]:
split_streets / (split_streets + complete_streets)

In [ ]:
complete_streets / (split_streets + complete_streets)

In [ ]:
# 78 percent of road miles are multi-segments streets
# 22 percent of road miles are single segment streets

# tabulate distances and distributions by street type

In [ ]:
st_type_df = pd.pivot_table(data = gdf, index = ['ord_street_type'],
                         columns = ['street_status_desc'], values = ['dist_miles'],
                        aggfunc='sum', fill_value = 0).reset_index()



In [ ]:
st_type_df.columns = [''.join(cn).replace('dist_miles', '') for cn in st_type_df.columns]

In [ ]:
st_type_df.shape

In [ ]:
st_type_df.head()

In [ ]:
# now, melt
st_type_df_melt = pd.melt(frame = st_type_df, id_vars = ['ord_street_type'],
                          var_name = 'street_status_desc',
                          value_name = 'total_miles')

In [ ]:
st_type_df_melt.head()

In [ ]:
col_names = ['ord_street_type', 'street_status_desc', 'dist_miles']

In [ ]:
ds_st_df = gdf[col_names].groupby(col_names[:-1]).describe().reset_index()

In [ ]:
ds_st_df.head()

In [ ]:
ds_st_df.columns =  [''.join(cn).replace('dist_miles', '') for cn in ds_st_df.columns]

In [ ]:
ds_st_df.head()

In [ ]:
temp_ds_st_df = ds_st_df.loc[ds_st_df['street_status_desc'] == 'added_street', :].copy()

In [ ]:
temp_ds_st_df = temp_ds_st_df.drop(labels = 'std', axis = 1)

In [ ]:
temp_ds_st_df = temp_ds_st_df.sort_values(by = ['count'], ascending = False)

In [ ]:
temp_ds_st_df.to_excel(excel_writer=e_writer, sheet_name = 'added_street_count', index = False)

In [ ]:
st_type_df_melt.head()

In [ ]:
ds_st_df.head()

In [ ]:
# join to get total miles and summary statistics
ds_st_df = pd.merge(left = st_type_df_melt, right = ds_st_df, how = 'left')

In [ ]:
ds_st_df.head()

In [ ]:
ds_st_df.columns

In [ ]:
for cn in ds_st_df.columns:
    if ds_st_df[cn].dtype == 'float64':
        ds_st_df[cn] = ds_st_df[cn].fillna(0)

In [ ]:
ds_st_df = ds_st_df.sort_values(by = ['ord_street_type', 'street_status_desc'])

In [ ]:
ds_st_df['street_status_desc'] = ds_st_df['street_status_desc'].str.replace('_street', '')

In [ ]:
# set index

In [ ]:
ds_st_df = ds_st_df.set_index(keys = ['ord_street_type', 'street_status_desc'])

In [ ]:
# reorder columns
col_names = ['count', 'min', '25%', '50%', 'mean', '75%', 'max', 'total_miles']
ds_st_df = ds_st_df[col_names]

In [ ]:
ds_st_df.to_excel(excel_writer=e_writer, sheet_name = 'sum_stats', index = True)
                  

In [ ]:
e_writer.close()

In [ ]:
ds_st_df.head()

# 10 longest added streets

In [ ]:
wms_gdf = gdf.loc[gdf['street_status'] == 2, :].copy()

In [ ]:
wms_gdf['dist_rank'] = wms_gdf['dist'].rank(method = 'dense', ascending = False)

In [ ]:
wms_gdf.loc[wms_gdf['dist_rank'] <= 10, 'ord_stname_concat'].tolist()

# histogram of all added streets

In [ ]:
# log transform the distance in miles to prep for plotting
wms_gdf['dist_miles_log'] = np.log10(wms_gdf['dist_miles'])

In [ ]:
wms_gdf['dist_miles_log'].describe()

In [ ]:
# compute summary stats and create a dataframe
# the ds_df will hold values of interest to plot and plotting keywords for 
# those values
ds_df = wms_gdf['dist_miles'].describe().to_frame().reset_index(names = ['stat'])

In [ ]:
# remove the count and the standard deviation - not useful for plotting
ds_df = ds_df.loc[-ds_df['stat'].isin(['count', 'std']), :]

In [ ]:
# add the 10th, 95th, and 99th precentiles
temp_stat_records = [
    ['10%', np.quantile(a = wms_gdf['dist_miles'], q = 0.10)],
    ['95%', np.quantile(a = wms_gdf['dist_miles'], q = 0.95)],
    ['99%', np.quantile(a = wms_gdf['dist_miles'], q = 0.99)]
]
temp_stat_df = pd.DataFrame(data = temp_stat_records, columns = ds_df.columns)
temp_stat_df.head()

In [ ]:
# combine
ds_df = pd.concat(objs = [ds_df, temp_stat_df], axis = 0)

In [ ]:
ds_df = ds_df.sort_values(by = ['dist_miles'])

In [ ]:
ds_df['dist_miles_log'] = ds_df['dist_miles'].map(lambda x: np.log10(x))

In [ ]:
# the color of the lines to add
ds_df['color_list'] = 'black'

In [ ]:
# convert to feet to help with labeling
ds_df['dist_feet'] = ds_df['dist_miles'] * 5280

In [ ]:
# this the horizontal alignment of each piece of added text
ds_df['text_ha'] = ['left', 'right',  'center', 'right', 'right', 'left', 'center', 'center', 'right']

In [ ]:
# the text labels to add
ds_df['text'] = ['4ft (min.)', '99 ft (10%)', '172 ft (25%)', '443 ft (med.)', '0.23 mi (avg.)', '0.25 mi (75%)', '0.96 mi (95%)','1.8 mi (99%)', '5 mi (max.)']

In [ ]:
# jitter each piece of text by a small amount left or right
ds_df['text_jitter'] = [0, .001, 0, .09, 0, 0, -.05, 0, 0]

In [ ]:
# the horizontal position is created from the distance in miles and the text jitter
ds_df['text_x_pos'] = ds_df['dist_miles_log'] + ds_df['text_jitter']

In [ ]:
ds_df.head(n=10)

In [ ]:
wms_gdf.head()

In [ ]:
# compute the histogram of the added distribution
sns.set_theme(style = "whitegrid")
f, ax = plt.subplots(figsize = (20, 5))

ax.set_ylim(0, 185.6)
ax.set_xlim(-3.2, .75)

my_plot = sns.histplot(data = wms_gdf, x = 'dist_miles_log',
                      color='#ca0020', bins = 100)

# set the top part of the plot frame - the spine - to black
ax.spines['top'].set_edgecolor(color = 'black')

y_ticks = list(range(0, 176, 25))
y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   

my_plot.set_yticks(ticks = y_ticks)
my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)

# total added streets
n_added = wms_gdf.shape[0]
n_added =  f"{n_added:,}"

tot_miles = wms_gdf['dist_miles'].sum()
tot_miles = f"{int(round(tot_miles, 0)):,}"

my_title = "Histogram of added road segment length, all road types (n = {} | {} miles)".format(n_added, tot_miles)

plt.title(label = my_title, fontsize = 16)
plt.xlabel(xlabel = 'Added road segment length (log-scale)')
plt.ylabel(ylabel = "Number of misssing road segments")

# x tick positions
x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1, 2, 3]
x_ticks = [np.log10(xtl) for xtl in x_tick_labels]

# x tick labels
x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi', '2 mi', '3 mi']


my_plot.set_xticks(ticks = x_ticks)
my_plot.set_xticklabels(labels = x_tick_label_text, rotation=0)

# white out the top portion of the ticks
for xt in x_ticks:
    my_plot.vlines(x = xt, ymin = 175, ymax = 185, color = 'white', linestyles = 'solid', linewidth = 2)

# add vertical lines at some descriptive statistics points
my_plot.vlines(x = ds_df['dist_miles_log'], ymin = 0, ymax = 175, color = ds_df['color_list'], linestyles = 'dashdot')

# add text for the descriptive stats
for ir, row in ds_df.iterrows():
    my_plot.text(x = row['text_x_pos'], y = 176, s = row['text'], horizontalalignment = row['text_ha'], color = '#ca0020')

output_file_name = '..\\graphics\\histogram_ALL_streets.png'
output_file_name = os.path.normpath(output_file_name)
print(output_file_name)
my_plot.get_figure().savefig(fname = output_file_name)
plt.show()

In [ ]:
# and there you have it... most added streets are short. 
# 75% of all added streets are 0.25 miles or less. 

# histogram of each type of street

In [ ]:
st_type_df = pd.pivot_table(data = wms_gdf, index = ['ord_street_type'],
                         columns = ['street_status_desc'], values = ['dist_miles'],
                        aggfunc='sum', fill_value = 0).reset_index()

In [ ]:
st_type_df.head()

In [ ]:
st_type_df.columns = [''.join(cn).replace('dist_miles', '') for cn in st_type_df.columns]

In [ ]:
st_type_df.head()

In [ ]:
st_type_df.columns = ['ord_street_type', 'total_miles']

In [ ]:
ds_df = wms_gdf[['ord_street_type', 'dist_miles']].groupby(['ord_street_type'])['dist_miles'].describe().reset_index(names = ['ord_street_type'])

In [ ]:
ds_df.head()

In [ ]:
ds_df['max'].tolist()

In [ ]:
ds_df.shape

In [ ]:
for ir, row in ds_df.iterrows():
    # let's make a graphic showing the distances
    sns.set_theme(style = "whitegrid")
    f, ax = plt.subplots(figsize = (20, 5))

    # set the top part of the plot frame - the spine - to black
    ax.spines['top'].set_edgecolor(color = 'black')

    x_max = row['max']
    ost = row['ord_street_type']    
    
    if x_max > 1:
        xlim_max = .75
    else:
        xlim_max = 0
    ax.set_xlim(-3.2, xlim_max)

    temp_gdf = wms_gdf.loc[wms_gdf['ord_street_type'] == ost, :]
    my_plot = sns.histplot(data = temp_gdf, x = 'dist_miles_log',
                          color='#ca0020', bins = 100)
    
    # total added streets
    n_added = temp_gdf.shape[0]
    n_added =  f"{n_added:,}"

    tot_miles = temp_gdf['dist_miles'].sum()    
    dist_units = 'miles'
    if tot_miles < .25:
        tot_dist = int(round(temp_gdf['dist'].sum(), 0))
        dist_units = 'feet'        
    elif tot_miles >= .25 and tot_miles <= 5:    
        tot_dist = tot_miles
        tot_dist = round(tot_dist, 2)        
    else:
        tot_dist = int(round(tot_miles, 0))        
        
    my_title =   "Histogram of added road segment length: {} (n = {} | {} {})".format(ost, n_added, tot_dist, dist_units )
    
    plt.title(label = my_title, fontsize = 16)
    plt.xlabel(xlabel = 'Added road segment  length (log-scale)')
    plt.ylabel(ylabel = "Number of misssing road segments")
    
    if x_max > 1:
        x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1, 2, 3]
        x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi', '2 mi', '3 mi']
    else:
        x_tick_labels = [50 / 5280, 100 / 5280, 300 / 5280, .1, .25, .5, 1]
        x_tick_label_text = ['50 ft', '100 ft', '300 ft',  '1/10 mi', '1/4 mi', '1/2 mi', '1 mi']
        
    x_ticks = [np.log10(xtl) for xtl in x_tick_labels]
    
    my_plot.set_xticks(ticks = x_ticks)
    my_plot.set_xticklabels(labels = x_tick_label_text, rotation=0)    
    
    
    output_file_name = '..\\graphics\\histogram_{}.png'.format(row['ord_street_type'])
    output_file_name = os.path.normpath(output_file_name)
    print(output_file_name)
    my_plot.get_figure().savefig(fname = output_file_name)
    plt.close()

In [ ]:
ds_st_df.head()

In [ ]:
col_names = ['ord_street_type', 'street_status_desc', 'dist_miles']
ds_st_df = gdf[col_names].groupby(col_names[:-1]).agg(n_segments = ('dist_miles', 'size'),
                                                      n_miles = ('dist_miles', 'sum')).reset_index()

In [ ]:
ds_st_df.head()

In [ ]:
col_names = ['street_status_desc', 'dist_miles']
all_ds_st_df = gdf[col_names].groupby(col_names[:-1]).agg(n_segments = ('dist_miles', 'size'),
                                                      n_miles = ('dist_miles', 'sum')).reset_index()

In [ ]:
all_ds_st_df['ord_street_type'] = 'ALL'

In [ ]:
col_names = ['ord_street_type', 'street_status_desc', 'n_segments', 'n_miles']
all_ds_st_df = all_ds_st_df[col_names]

In [ ]:
ds_st_df = pd.concat(objs = [all_ds_st_df, ds_st_df])

In [ ]:
ds_st_df.head()

In [ ]:
ds_st_df['n_miles_log'] = np.log(ds_st_df['n_miles'])
ds_st_df['street_status_desc'] = ds_st_df['street_status_desc'].str.replace('_', ' ')

In [ ]:
sns.set_theme(style = "whitegrid")
f, ax = plt.subplots(figsize = (20, 5))
ax.set_ylim(0, 1600)

# these are the same green, black, and red colors used in qGIS. 
my_color_palette = {'continuous street': '#33a02c',
'discontinuous street': '#000000',
'added street': '#ca0020'}

my_plot = sns.barplot(data = ds_st_df, x = 'ord_street_type', y = 'n_miles',
                      hue = 'street_status_desc', palette=my_color_palette)

# set the top part of the plot frame - the spine - to black
ax.spines['top'].set_edgecolor(color = 'black')

y_ticks = list(range(0, 1601, 200))
y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   

my_plot.set_yticks(ticks = y_ticks)
my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)

my_title = "Total miles by road types and road status"

plt.title(label = my_title, fontsize = 16)
plt.xlabel(xlabel = 'Road type')
plt.ylabel(ylabel = "Miles")

plt.legend(title='Road status')

output_file_name = '..\\graphics\\barplot_miles.png'
output_file_name = os.path.normpath(output_file_name)
print(output_file_name)
my_plot.get_figure().savefig(fname = output_file_name)
plt.show()

In [ ]:
sns.set_theme(style = "whitegrid")
f, ax = plt.subplots(figsize = (20, 5))
ax.set_ylim(0, 22000)

my_plot = sns.barplot(data = ds_st_df, x = 'ord_street_type', y = 'n_segments',
                      hue = 'street_status_desc', palette=my_color_palette)

# set the top part of the plot frame - the spine - to black
ax.spines['top'].set_edgecolor(color = 'black')

y_ticks = list(range(0, 22001, 2000))
y_tick_labels_formatted = ['{:,}'.format(ytl) for ytl in y_ticks]   

my_plot.set_yticks(ticks = y_ticks)
my_plot.set_yticklabels(labels = y_tick_labels_formatted, rotation=0)

my_title = "Total segments by road types and road status"

plt.title(label = my_title, fontsize = 16)
plt.xlabel(xlabel = 'Road type')
plt.ylabel(ylabel = "Number of segments")

plt.legend(title='Road status')

output_file_name = '..\\graphics\\barplot_segment_count.png'
output_file_name = os.path.normpath(output_file_name)
print(output_file_name)
my_plot.get_figure().savefig(fname = output_file_name)
plt.show()